In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [2]:
data = pd.read_csv('creditcard.csv', na_filter=True)

data.drop(['Time'], axis=1, inplace=True)
selected_vars =['V4','V14','V10','V16','V9','V12','V11','V17','V7','V21','V18','V8','V1','V3','V6','V2','V27','V5','V26','Class']


In [3]:
tr_data = data[selected_vars]
tr_data.shape

(284807, 20)

In [4]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [5]:
y=tr_data['Class']

In [6]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
X=pd.DataFrame(StandardScaler().fit_transform(X))

In [7]:
X_train0,X_test,y_train0,y_test = train_test_split(X, y, test_size=0.3, shuffle=False,random_state=0)
X_train0,X_train3,y_train0,y_train3 = train_test_split(X_train0, y_train0, test_size=0.25, shuffle=False,random_state=0)
X_train0,X_train2,y_train0,y_train2 = train_test_split(X_train0, y_train0, test_size=0.333, shuffle=False,random_state=0)
X_train0,X_train1,y_train0,y_train1 = train_test_split(X_train0, y_train0, test_size=0.5, shuffle=False,random_state=0)

from sklearn.linear_model import SGDOneClassSVM
df1_normal=X_train0[y_train0==0]
clf = SGDOneClassSVM(nu=0.024,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train0.drop(X_train0.index[index], inplace=True)
y_train0.drop(y_train0.index[index], inplace=True)

df1_normal=X_train1[y_train1==1]
clf = SGDOneClassSVM(nu=0.039,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.95)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train1.drop(X_train1.index[index], inplace=True)
y_train1.drop(y_train1.index[index], inplace=True)

df1_normal=X_train2[y_train2==1]
clf = SGDOneClassSVM(nu=0.085,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.90)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train2.drop(X_train2.index[index], inplace=True)
y_train2.drop(y_train2.index[index], inplace=True)

df1_normal=X_train3[y_train3==1]
clf = SGDOneClassSVM(nu=0.1,random_state=0)
res = clf.fit_predict(df1_normal)
res[res==1] = 0
res[res==-1] = 1
ros = RandomUnderSampler(sampling_strategy=0.90)
ros.fit_resample(df1_normal,res)
index=ros.sample_indices_
X_train3.drop(X_train3.index[index], inplace=True)
y_train3.drop(y_train3.index[index], inplace=True)

In [8]:
X_train0.shape,X_train1.shape,X_train2.shape,X_train3.shape

((49139, 19), (49864, 19), (49790, 19), (49839, 19))

In [9]:
y_train0.shape,y_train1.shape

((49139,), (49864,))

In [10]:
X_test.shape

(85443, 19)

# N=1

In [11]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X0=X_train0.to_numpy()
test_LSTM_X=X_test.to_numpy()
train_LSTM_y0=y_train0.to_numpy()
test_LSTM_y=y_test.to_numpy()
w=3
X_train0=np.zeros([train_LSTM_X0.shape[0]-w+1,w, train_LSTM_X0.shape[1]])
for i in range(train_LSTM_X0.shape[0]-w+1):
    for j in range(w):
        X_train0[i][j]=train_LSTM_X0[i+j]
X_test=np.zeros([test_LSTM_X.shape[0]-w+1,w, test_LSTM_X.shape[1]])
for i in range(test_LSTM_X.shape[0]-w+1):
    for j in range(w):
        X_test[i][j]=test_LSTM_X[i+j]
## Recuperation des labels

y_train0=np.delete(train_LSTM_y0,[*range(0, w-1, 1)], 0)
y_test=np.delete(test_LSTM_y,[*range(0, w-1, 1)], 0)

train_LSTM_X0=X_train0
test_LSTM_X=X_test
train_LSTM_y0=y_train0
test_LSTM_y=y_test

In [12]:
inputs=Input((w,19))
x1=LSTM(100)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model0=Model(inputs,outputs)

In [13]:
model0.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [14]:
history0=model0.fit(train_LSTM_X0, train_LSTM_y0,epochs=4,batch_size=200,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/4
246/246 [==============================] - 7s 18ms/step - loss: 0.0790 - precision: 0.0808 - recall: 0.3378 - val_loss: 0.1236 - val_precision: 0.0248 - val_recall: 0.5556
Epoch 2/4
246/246 [==============================] - 4s 15ms/step - loss: 0.0051 - precision: 0.7718 - recall: 0.7770 - val_loss: 0.0459 - val_precision: 0.4904 - val_recall: 0.7130
Epoch 3/4
246/246 [==============================] - 4s 15ms/step - loss: 0.0040 - precision: 0.7785 - recall: 0.8311 - val_loss: 0.0224 - val_precision: 0.9111 - val_recall: 0.7593
Epoch 4/4
246/246 [==============================] - 4s 15ms/step - loss: 0.0034 - precision: 0.7927 - recall: 0.8784 - val_loss: 0.0130 - val_precision: 0.9535 - val_recall: 0.7593


In [15]:
# predict probabilities for test set
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs0 = yhat_probs0[:, 0]> 0.5

In [16]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs0)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs0)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs0)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs0)
print('f1: %f' % f1)

Accuracy: 0.999649
Precision: 0.953488
Recall: 0.759259
f1: 0.845361


# N=2

In [17]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X1=X_train1.to_numpy()
train_LSTM_y1=y_train1.to_numpy()
w=3
X_train1=np.zeros([train_LSTM_X1.shape[0]-w+1,w, train_LSTM_X1.shape[1]])
for i in range(train_LSTM_X1.shape[0]-w+1):
    for j in range(w):
        X_train1[i][j]=train_LSTM_X1[i+j]

y_train1=np.delete(train_LSTM_y1,[*range(0, w-1, 1)], 0)
train_LSTM_X1=X_train1
train_LSTM_y1=y_train1

In [18]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs,outputs)

In [19]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [20]:
history1=model1.fit(train_LSTM_X1, train_LSTM_y1,epochs=6,batch_size=200,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/6
250/250 [==============================] - 6s 13ms/step - loss: 0.1100 - precision: 0.0052 - recall: 0.0533 - val_loss: 0.1396 - val_precision: 0.0155 - val_recall: 0.3796
Epoch 2/6
250/250 [==============================] - 3s 10ms/step - loss: 0.0056 - precision: 0.9167 - recall: 0.2933 - val_loss: 0.0661 - val_precision: 0.1410 - val_recall: 0.6019
Epoch 3/6
250/250 [==============================] - 2s 10ms/step - loss: 0.0043 - precision: 0.9592 - recall: 0.6267 - val_loss: 0.0380 - val_precision: 0.5909 - val_recall: 0.7222
Epoch 4/6
250/250 [==============================] - 3s 10ms/step - loss: 0.0039 - precision: 0.9455 - recall: 0.6933 - val_loss: 0.0284 - val_precision: 0.8020 - val_recall: 0.7500
Epoch 5/6
250/250 [==============================] - 3s 11ms/step - loss: 0.0036 - precision: 0.9153 - recall: 0.7200 - val_loss: 0.0208 - val_precision: 0.9111 - val_recall: 0.7593
Epoch 6/6
250/250 [==============================] - 3s 11ms/step - loss: 0.0034 - precisi

In [21]:
# predict probabilities for test set
yhat_probs1 = model1.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs1 = yhat_probs1[:, 0]> 0.5

In [22]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs1)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs1)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs1)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs1)
print('f1: %f' % f1)

Accuracy: 0.999602
Precision: 0.920455
Recall: 0.750000
f1: 0.826531


# N=3

In [23]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X2=X_train2.to_numpy()
train_LSTM_y2=y_train2.to_numpy()
w=3
X_train2=np.zeros([train_LSTM_X2.shape[0]-w+1,w, train_LSTM_X2.shape[1]])
for i in range(train_LSTM_X2.shape[0]-w+1):
    for j in range(w):
        X_train2[i][j]=train_LSTM_X2[i+j]

y_train2=np.delete(train_LSTM_y2,[*range(0, w-1, 1)], 0)
train_LSTM_X2=X_train2
train_LSTM_y2=y_train2

In [24]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model2=Model(inputs,outputs)

In [25]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [26]:
history2=model2.fit(train_LSTM_X2, train_LSTM_y2,epochs=15,batch_size=200,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/15
249/249 [==============================] - 7s 16ms/step - loss: 0.1204 - precision: 0.0031 - recall: 0.0462 - val_loss: 0.0136 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
249/249 [==============================] - 3s 12ms/step - loss: 0.0053 - precision: 0.8750 - recall: 0.2154 - val_loss: 0.0065 - val_precision: 1.0000 - val_recall: 0.1296
Epoch 3/15
249/249 [==============================] - 3s 10ms/step - loss: 0.0043 - precision: 0.9355 - recall: 0.4462 - val_loss: 0.0045 - val_precision: 1.0000 - val_recall: 0.3611
Epoch 4/15
249/249 [==============================] - 2s 10ms/step - loss: 0.0038 - precision: 0.9459 - recall: 0.5385 - val_loss: 0.0040 - val_precision: 0.9815 - val_recall: 0.4907
Epoch 5/15
249/249 [==============================] - 3s 12ms/step - loss: 0.0035 - precision: 0.9512 - recall: 0.6000 - val_loss: 0.0039 - val_precision: 0.9841 - val_recall: 0.5741
Epoch 6/15
249/249 [==============================] - 3s 10ms/step - loss: 0.

In [27]:
# predict probabilities for test set
yhat_probs2 = model2.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs2 = yhat_probs2[:, 0]> 0.5

In [28]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs2)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs2)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs2)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs2)
print('f1: %f' % f1)

Accuracy: 0.999661
Precision: 1.000000
Recall: 0.731481
f1: 0.844920


# N=4

In [29]:
# X_train et X_test sont des dataframe qui contient les features
train_LSTM_X3=X_train3.to_numpy()
train_LSTM_y3=y_train3.to_numpy()
w=3
X_train3=np.zeros([train_LSTM_X3.shape[0]-w+1,w, train_LSTM_X3.shape[1]])
for i in range(train_LSTM_X3.shape[0]-w+1):
    for j in range(w):
        X_train3[i][j]=train_LSTM_X3[i+j]

y_train3=np.delete(train_LSTM_y3,[*range(0, w-1, 1)], 0)
train_LSTM_X3=X_train3
train_LSTM_y3=y_train3

In [30]:
inputs=Input((w,19))
x1=LSTM(50)(inputs)
outputs=Dense(1,activation='sigmoid')(x1)
model3=Model(inputs,outputs)

In [31]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])

In [32]:
history3=model3.fit(train_LSTM_X3, train_LSTM_y3,epochs=11,batch_size=200,validation_data=(test_LSTM_X, test_LSTM_y))

Epoch 1/11
250/250 [==============================] - 5s 12ms/step - loss: 0.1049 - precision: 0.0552 - recall: 0.4062 - val_loss: 0.0089 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/11
250/250 [==============================] - 2s 9ms/step - loss: 0.0062 - precision: 0.9412 - recall: 0.5000 - val_loss: 0.0051 - val_precision: 1.0000 - val_recall: 0.2315
Epoch 3/11
250/250 [==============================] - 2s 10ms/step - loss: 0.0043 - precision: 0.8986 - recall: 0.6458 - val_loss: 0.0039 - val_precision: 1.0000 - val_recall: 0.4722
Epoch 4/11
250/250 [==============================] - 2s 10ms/step - loss: 0.0035 - precision: 0.8987 - recall: 0.7396 - val_loss: 0.0034 - val_precision: 1.0000 - val_recall: 0.5648
Epoch 5/11
250/250 [==============================] - 2s 9ms/step - loss: 0.0031 - precision: 0.9024 - recall: 0.7708 - val_loss: 0.0031 - val_precision: 1.0000 - val_recall: 0.6111
Epoch 6/11
250/250 [==============================] - 2s 10ms/step - loss: 0.00

In [33]:
# predict probabilities for test set
yhat_probs3 = model3.predict(test_LSTM_X, verbose=0)
# reduce to 1d array
yhat_probs3 = yhat_probs3[:, 0]> 0.5

In [34]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, yhat_probs3)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, yhat_probs3)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, yhat_probs3)
print('Recall: %f' % recall)
f1 = f1_score(test_LSTM_y, yhat_probs3)
print('f1: %f' % f1)

Accuracy: 0.999625
Precision: 1.000000
Recall: 0.703704
f1: 0.826087


# FFNN

In [35]:
y_probs0 = model0.predict(train_LSTM_X0, verbose=0)
y_probs1 = model1.predict(train_LSTM_X0, verbose=0)
y_probs2 = model2.predict(train_LSTM_X0, verbose=0)
y_probs3 = model3.predict(train_LSTM_X0, verbose=0)

y_probs00 = model0.predict(train_LSTM_X1, verbose=0)
y_probs11 = model1.predict(train_LSTM_X1, verbose=0)
y_probs22 = model2.predict(train_LSTM_X1, verbose=0)
y_probs33 = model3.predict(train_LSTM_X1, verbose=0)

In [36]:
X_train0=np.concatenate((y_probs0,y_probs1,y_probs2,y_probs3),axis=1)
X_train1=np.concatenate((y_probs00,y_probs11,y_probs22,y_probs33),axis=1)
X_train=np.concatenate((X_train0,X_train1),axis=0)
y_train=np.concatenate((y_train0,y_train1),axis=0)
X_train.shape,y_train.shape

((98999, 4), (98999,))

In [37]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier().fit(X_train, y_train)

# evaluation

In [38]:
yhat_probs0 = model0.predict(test_LSTM_X, verbose=0)
yhat_probs1 = model1.predict(test_LSTM_X,verbose=0)
yhat_probs2 = model2.predict(test_LSTM_X,verbose=0)
yhat_probs3 = model3.predict(test_LSTM_X,verbose=0)
X_test=np.concatenate((yhat_probs0,yhat_probs1,yhat_probs2,yhat_probs3),axis=1)
y_pred=clf.predict(X_test)

In [39]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_LSTM_y, y_pred)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_LSTM_y, y_pred)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_LSTM_y, y_pred)
print('Recall: %f' % recall)
#F1 = 2 * (precision * recall) / (precision + recall)
f1=f1_score(test_LSTM_y, y_pred)
print('f1: %f' % f1)

Accuracy: 0.999637
Precision: 0.942529
Recall: 0.759259
f1: 0.841026
